In [ ]:
# Import required libraries and the utility function
from utils.qdrant_utils import get_qdrant_client
import pandas as pd


In [ ]:
# Connect to Qdrant using the utility function
client = get_qdrant_client()
if client:
    print("Connected to Qdrant!")
else:
    print("Failed to connect to Qdrant.")

In [ ]:
from typing import Any, Dict
from utils.job_utils import send_and_await_job

html_content = "<html><body><h1>Notebook Ingest</h1><p>Hello from the notebook.</p></body></html>"

payload: Dict[str, Any] = {
    "content": html_content,
    "resourceId": 123456,
    "projectId": 78910,
}

status, result = send_and_await_job("ingest-content", "high", payload)
print("Job status:", status)
print("Job result:", result)


In [ ]:
# Fetch current documents (points) from the Qdrant collection and show a summary table.

try:
    resp, next_page = client.scroll(collection_name="rag_docs", limit=10, with_payload=True, with_vectors=False)
except Exception as e:
    print("Error while scrolling collection:", e)
    resp = []

data = []
for point in resp:
    data.append({
        "id": point.id,
        "source_id": point.payload.get("source_id", None),
        "project_id": point.payload.get("project_id", None),
        "text": point.payload.get("text", ""),
        "part_number": point.payload.get("part_number", None),
    })

df = pd.DataFrame(data)
df